In [1]:
from util.PathConfig import path_config
import sqlite3
import pandas as pd
import numpy as np

C:\Users\Ethan\glucose-prediction\app\src\config/local_project_dirs.json
Loading config


In [2]:
# Function to insert row in the dataframe. From GeeksForGeeks.
def insert_row(row_number, df, row_value):
    # Starting value of upper half
    start_upper = 0
  
    # End value of upper half
    end_upper = row_number
  
    # Start value of lower half
    start_lower = row_number
  
    # End value of lower half
    end_lower = df.shape[0]
  
    # Create a list of upper_half index
    upper_half = [*range(start_upper, end_upper, 1)]
  
    # Create a list of lower_half index
    lower_half = [*range(start_lower, end_lower, 1)]
  
    # Increment the value of lower half by 1
    lower_half = [x.__add__(1) for x in lower_half]
  
    # Combine the two lists
    index_ = upper_half + lower_half
  
    # Update the index of the dataframe
    df.index = index_
  
    # Insert a row at the end
    df.loc[row_number] = row_value
   
    # Sort the index labels
    df = df.sort_index()
  
    # return the dataframe
    return df

In [3]:
db_dir = getattr(path_config, 'db_dir')
print(db_dir)

C:/Users/Ethan/glucose-prediction/data/HealthData/DBs


In [4]:
conn = sqlite3.connect(db_dir + '/garmin_monitoring.db')

In [5]:
cur = conn.cursor()

In [6]:
cur.execute("SELECT * FROM monitoring_hr WHERE timestamp like '2021-08-17%'")

In [7]:
heart_rate = cur.fetchall()

In [8]:
for row in heart_rate:
    print(row)

('2021-08-17 00:01:00.000000', 53)
('2021-08-17 00:02:00.000000', 52)
('2021-08-17 00:05:00.000000', 51)
('2021-08-17 00:06:00.000000', 52)
('2021-08-17 00:07:00.000000', 51)
('2021-08-17 00:09:00.000000', 52)
('2021-08-17 00:16:00.000000', 51)
('2021-08-17 00:18:00.000000', 52)
('2021-08-17 00:20:00.000000', 51)
('2021-08-17 00:22:00.000000', 52)
('2021-08-17 00:23:00.000000', 51)
('2021-08-17 00:27:00.000000', 52)
('2021-08-17 00:28:00.000000', 53)
('2021-08-17 00:29:00.000000', 58)
('2021-08-17 00:30:00.000000', 56)
('2021-08-17 00:31:00.000000', 54)
('2021-08-17 00:32:00.000000', 57)
('2021-08-17 00:33:00.000000', 59)
('2021-08-17 00:34:00.000000', 57)
('2021-08-17 00:35:00.000000', 56)
('2021-08-17 00:36:00.000000', 57)
('2021-08-17 00:37:00.000000', 56)
('2021-08-17 00:38:00.000000', 57)
('2021-08-17 00:39:00.000000', 55)
('2021-08-17 00:41:00.000000', 54)
('2021-08-17 00:42:00.000000', 55)
('2021-08-17 00:44:00.000000', 56)
('2021-08-17 00:45:00.000000', 54)
('2021-08-17 00:46:0

In [9]:
hr_df = pd.read_sql_query("SELECT * FROM monitoring_hr", conn)

In [10]:
hr_df.head()

,timestamp,heart_rate
0,2021-08-15 00:02:00.000000,47
1,2021-08-15 00:04:00.000000,45
2,2021-08-15 00:05:00.000000,49
3,2021-08-15 00:06:00.000000,48
4,2021-08-15 00:07:00.000000,46


In [11]:
hr_df

,timestamp,heart_rate
0,2021-08-15 00:02:00.000000,47
1,2021-08-15 00:04:00.000000,45
2,2021-08-15 00:05:00.000000,49
3,2021-08-15 00:06:00.000000,48
4,2021-08-15 00:07:00.000000,46
...,...,...
48959,2021-09-26 13:11:00.000000,72
48960,2021-09-26 13:12:00.000000,84
48961,2021-09-26 13:13:00.000000,76
48962,2021-09-26 13:14:00.000000,81


In [12]:
hr_df.describe()

,heart_rate
count,48964.000000
mean,70.186055
std,18.647615
min,43.000000
25%,58.000000
50%,67.000000
75%,77.000000
max,194.000000


In [13]:
hr_df.shape

(48964, 2)

In [14]:
hr_df.columns

Index(['timestamp', 'heart_rate'], dtype='object')

In [15]:
hr_df.dtypes

timestamp     object
heart_rate     int64
dtype: object

In [16]:
hr_df.isna().sum()

timestamp     0
heart_rate    0
dtype: int64

In [17]:
# Convert all the timestamps to datetimes
hr_df['timestamp'] = pd.to_datetime(hr_df['timestamp'], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
# Assert that all timestamps were converted successfully
assert hr_df.timestamp.isnull().sum() == 0, 'timestamp conversion failed'

In [18]:
hr_df.dtypes

timestamp     datetime64[ns]
heart_rate             int64
dtype: object

In [19]:
start = pd.to_datetime(str(hr_df['timestamp'].min()))
end = pd.to_datetime(str(hr_df['timestamp'].max()))

In [20]:
print(start)

2021-08-15 00:02:00


In [21]:
print(end)

2021-09-26 13:15:00


In [22]:
full_timestamps = pd.date_range(start=start, end=end, freq='1Min')

In [23]:
# To fill in empty timestamps, reindex the dataframe with the the filled in datetime range and reindex to get rid of the old
# timestamp column. This should automatically populate the dataframe with NaN values. Reset the index again to make the
# index column into a regular column, and rename it back from 'index' to 'timestamp'.
hr_df = hr_df.set_index('timestamp').reindex(full_timestamps).rename_axis('timestamp').reset_index()


# I'm not sure why, but calling reindex inline and chaining off of reset_index() seems to cause the columns list to still have
# the old ones from before the initial reindexing.
hr_df.reindex(columns=hr_df.columns)

,timestamp,heart_rate
0,2021-08-15 00:02:00,47.0
1,2021-08-15 00:03:00,NaN
2,2021-08-15 00:04:00,45.0
3,2021-08-15 00:05:00,49.0
4,2021-08-15 00:06:00,48.0
...,...,...
61269,2021-09-26 13:11:00,72.0
61270,2021-09-26 13:12:00,84.0
61271,2021-09-26 13:13:00,76.0
61272,2021-09-26 13:14:00,81.0


In [24]:
hr_df.head()

,timestamp,heart_rate
0,2021-08-15 00:02:00,47.0
1,2021-08-15 00:03:00,NaN
2,2021-08-15 00:04:00,45.0
3,2021-08-15 00:05:00,49.0
4,2021-08-15 00:06:00,48.0


In [25]:
hr_df.isna().sum()

timestamp         0
heart_rate    12342
dtype: int64

In [26]:
dex_dir = getattr(path_config, 'dexcom_data_dir')
print(dex_dir)

C:/Users/Ethan/glucose-prediction/data/dexcom


In [27]:
dex_df = pd.read_csv(dex_dir + "/CLARITY_Export__Levi_2021-09-16_032109.csv")

In [28]:
dex_df = dex_df.rename(columns={'Timestamp (YYYY-MM-DDThh:mm:ss)': 'timestamp'})

In [29]:
dex_df.head()

,Index,timestamp,Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
0,1,NaN,FirstName,NaN,Levi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,LastName,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,Device,NaN,NaN,Dexcom Receiver with G6,PL84498928,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,Alert,Fall,NaN,NaN,PL84498928,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,5,NaN,Alert,High,NaN,NaN,PL84498928,180.0,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
dex_df = dex_df[dex_df['timestamp'].notnull()]

In [31]:
dex_df.head(15)

,Index,timestamp,Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
10,11,2021-09-14T00:04:25,EGV,NaN,NaN,NaN,PL84498928,143.0,NaN,NaN,NaN,NaN,8779273.0,8N8YQB
11,12,2021-09-14T00:09:25,EGV,NaN,NaN,NaN,PL84498928,140.0,NaN,NaN,NaN,NaN,8779573.0,8N8YQB
12,13,2021-09-14T00:14:25,EGV,NaN,NaN,NaN,PL84498928,139.0,NaN,NaN,NaN,NaN,8779873.0,8N8YQB
13,14,2021-09-14T00:19:25,EGV,NaN,NaN,NaN,PL84498928,121.0,NaN,NaN,NaN,NaN,8780173.0,8N8YQB
14,15,2021-09-14T00:24:26,EGV,NaN,NaN,NaN,PL84498928,111.0,NaN,NaN,NaN,NaN,8780473.0,8N8YQB
15,16,2021-09-14T00:29:25,EGV,NaN,NaN,NaN,PL84498928,113.0,NaN,NaN,NaN,NaN,8780773.0,8N8YQB
16,17,2021-09-14T00:34:25,EGV,NaN,NaN,NaN,PL84498928,119.0,NaN,NaN,NaN,NaN,8781073.0,8N8YQB
17,18,2021-09-14T00:39:25,EGV,NaN,NaN,NaN,PL84498928,126.0,NaN,NaN,NaN,NaN,8781373.0,8N8YQB
18,19,2021-09-14T00:44:26,EGV,NaN,NaN,NaN,PL84498928,143.0,NaN,NaN,NaN,NaN,8781673.0,8N8YQB
19,20,2021-09-14T00:49:25,EGV,NaN,NaN,NaN,PL84498928,146.0,NaN,NaN,NaN,NaN,8781973.0,8N8YQB


In [32]:
# Convert all the timestamps to datetimes
dex_df['timestamp'] = pd.to_datetime(dex_df['timestamp'], format = '%Y-%m-%dT%H:%M:%S', errors = 'coerce')
# Assert that all timestamps were converted successfully
assert dex_df.timestamp.isnull().sum() == 0, 'timestamp conversion failed'

In [33]:
dex_df.head(15)

,Index,timestamp,Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
10,11,2021-09-14 00:04:25,EGV,NaN,NaN,NaN,PL84498928,143.0,NaN,NaN,NaN,NaN,8779273.0,8N8YQB
11,12,2021-09-14 00:09:25,EGV,NaN,NaN,NaN,PL84498928,140.0,NaN,NaN,NaN,NaN,8779573.0,8N8YQB
12,13,2021-09-14 00:14:25,EGV,NaN,NaN,NaN,PL84498928,139.0,NaN,NaN,NaN,NaN,8779873.0,8N8YQB
13,14,2021-09-14 00:19:25,EGV,NaN,NaN,NaN,PL84498928,121.0,NaN,NaN,NaN,NaN,8780173.0,8N8YQB
14,15,2021-09-14 00:24:26,EGV,NaN,NaN,NaN,PL84498928,111.0,NaN,NaN,NaN,NaN,8780473.0,8N8YQB
15,16,2021-09-14 00:29:25,EGV,NaN,NaN,NaN,PL84498928,113.0,NaN,NaN,NaN,NaN,8780773.0,8N8YQB
16,17,2021-09-14 00:34:25,EGV,NaN,NaN,NaN,PL84498928,119.0,NaN,NaN,NaN,NaN,8781073.0,8N8YQB
17,18,2021-09-14 00:39:25,EGV,NaN,NaN,NaN,PL84498928,126.0,NaN,NaN,NaN,NaN,8781373.0,8N8YQB
18,19,2021-09-14 00:44:26,EGV,NaN,NaN,NaN,PL84498928,143.0,NaN,NaN,NaN,NaN,8781673.0,8N8YQB
19,20,2021-09-14 00:49:25,EGV,NaN,NaN,NaN,PL84498928,146.0,NaN,NaN,NaN,NaN,8781973.0,8N8YQB


In [34]:
# Round Dexcom timestamps to the nearest minute to align with Garmin timestamps.
dex_df['timestamp'] = dex_df['timestamp'].round('min')

In [35]:
dex_df.head(15)

,Index,timestamp,Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
10,11,2021-09-14 00:04:00,EGV,NaN,NaN,NaN,PL84498928,143.0,NaN,NaN,NaN,NaN,8779273.0,8N8YQB
11,12,2021-09-14 00:09:00,EGV,NaN,NaN,NaN,PL84498928,140.0,NaN,NaN,NaN,NaN,8779573.0,8N8YQB
12,13,2021-09-14 00:14:00,EGV,NaN,NaN,NaN,PL84498928,139.0,NaN,NaN,NaN,NaN,8779873.0,8N8YQB
13,14,2021-09-14 00:19:00,EGV,NaN,NaN,NaN,PL84498928,121.0,NaN,NaN,NaN,NaN,8780173.0,8N8YQB
14,15,2021-09-14 00:24:00,EGV,NaN,NaN,NaN,PL84498928,111.0,NaN,NaN,NaN,NaN,8780473.0,8N8YQB
15,16,2021-09-14 00:29:00,EGV,NaN,NaN,NaN,PL84498928,113.0,NaN,NaN,NaN,NaN,8780773.0,8N8YQB
16,17,2021-09-14 00:34:00,EGV,NaN,NaN,NaN,PL84498928,119.0,NaN,NaN,NaN,NaN,8781073.0,8N8YQB
17,18,2021-09-14 00:39:00,EGV,NaN,NaN,NaN,PL84498928,126.0,NaN,NaN,NaN,NaN,8781373.0,8N8YQB
18,19,2021-09-14 00:44:00,EGV,NaN,NaN,NaN,PL84498928,143.0,NaN,NaN,NaN,NaN,8781673.0,8N8YQB
19,20,2021-09-14 00:49:00,EGV,NaN,NaN,NaN,PL84498928,146.0,NaN,NaN,NaN,NaN,8781973.0,8N8YQB


In [36]:
# Drop unnecessary columns.
dex_df.drop('Index', axis=1, inplace=True)
dex_df.drop('Event Type', axis=1, inplace=True)
dex_df.drop('Source Device ID', axis=1, inplace=True)
dex_df.drop('Event Subtype', axis=1, inplace=True)
dex_df.drop('Patient Info', axis=1, inplace=True)
dex_df.drop('Device Info', axis=1, inplace=True)
dex_df.drop('Glucose Rate of Change (mg/dL/min)', axis=1, inplace=True)
dex_df.drop('Transmitter Time (Long Integer)', axis=1, inplace=True)
dex_df.drop('Transmitter ID', axis=1, inplace=True)
dex_df.drop('Duration (hh:mm:ss)', axis=1, inplace=True)

dex_df.head()

,timestamp,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
10,2021-09-14 00:04:00,143.0,NaN,NaN
11,2021-09-14 00:09:00,140.0,NaN,NaN
12,2021-09-14 00:14:00,139.0,NaN,NaN
13,2021-09-14 00:19:00,121.0,NaN,NaN
14,2021-09-14 00:24:00,111.0,NaN,NaN


In [37]:
dex_df.reset_index(drop=True, inplace=True)
#dex_df.reindex(columns=dex_df.columns)
dex_df.head()

,timestamp,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
0,2021-09-14 00:04:00,143.0,NaN,NaN
1,2021-09-14 00:09:00,140.0,NaN,NaN
2,2021-09-14 00:14:00,139.0,NaN,NaN
3,2021-09-14 00:19:00,121.0,NaN,NaN
4,2021-09-14 00:24:00,111.0,NaN,NaN


In [38]:
dex_df[dex_df['timestamp'] == '2021-09-14 22:44:00']

,timestamp,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
225,2021-09-14 22:44:00,NaN,1.0,NaN


In [39]:
dex_df.shape

(502, 4)

In [40]:
# Inserting a glucose reading with the same timestamp as the above insulin event. I want to make sure I can combine these into
# one row.
dex_df = insert_row(224, dex_df, [pd.to_datetime('2021-09-14 22:44:00', format = '%Y-%m-%dT%H:%M:%S'), 143.0, np.nan, np.nan])
dex_df.shape

(503, 4)

In [41]:
dex_df[dex_df['timestamp'] == '2021-09-14 22:44:00']

,timestamp,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
224,2021-09-14 22:44:00,143.0,NaN,NaN
226,2021-09-14 22:44:00,NaN,1.0,NaN


In [42]:
# Combine any of these dexcom rows with same timestamp into one row.
#aggregation_functions = {'Glucose Value (mg/dL)': 'sum', 'Insulin Value (u)': 'sum', 'Carb Value (grams)': 'sum'}
#dex_df = dex_df.groupby(dex_df['timestamp']).aggregate(aggregation_functions)

In [43]:
#df_new[df_new['timestamp'] == '2021-09-14 22:44:00']
#dex_df[dex_df.index == '2021-09-14 22:44:00']

In [44]:
#dex_df['Insulin Value (u)'] = dex_df['Insulin Value (u)'].fillna(0)
#dex_df['Insulin Value (u)'] = dex_df['Insulin Value (u)'].fillna(0)

In [45]:
# Now we want to reassign all NaN values to the proper ones. For heart rate and glucose, this means we want to interpolate to
# fill in any gaps. For Insulin and Carbs we want all NaN values to go to 0. Note that NaN values are skipped by default, i.e.
# treated as 0 in 
#dex_df['Glucose Value (mg/dL)'] = dex_df['Glucose Value (mg/dL)'].interpolate(method='linear')
#dex_df['Insulin Value (u)'] = dex_df['Insulin Value (u)'].interpolate(method='linear')
#dex_df['Carb Value (grams)'] = dex_df['Carb Value (grams)'].interpolate(method='linear')

In [46]:
print("HR dataframe shape: ", hr_df.shape)
print("Dexcom dataframe shape: ", dex_df.shape)

HR dataframe shape:  (61274, 2)
Dexcom dataframe shape:  (503, 4)


In [47]:
merged_data = pd.merge(hr_df, dex_df, how='outer', on=['timestamp'])

In [48]:
# Combine any of these dexcom rows with same timestamp into one row. Note that the sum function ignores NaN values, so they
# will be treated as 0.
aggregation_functions = {'heart_rate': 'first', 'Glucose Value (mg/dL)': 'sum', 'Insulin Value (u)': 'sum', 
                         'Carb Value (grams)': 'sum'}
merged_data = merged_data.groupby(merged_data['timestamp']).aggregate(aggregation_functions)

In [51]:
merged_data.head()

,heart_rate,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
timestamp,,,,
2021-08-15 00:02:00,47.0,0.0,0.0,0.0
2021-08-15 00:03:00,NaN,0.0,0.0,0.0
2021-08-15 00:04:00,45.0,0.0,0.0,0.0
2021-08-15 00:05:00,49.0,0.0,0.0,0.0
2021-08-15 00:06:00,48.0,0.0,0.0,0.0


In [49]:
# Check to see that the carb/insulin events and glucose readings with the same timestamp have been properly combined along with
# their heart rate.
merged_data[merged_data.index == '2021-09-14 22:44:00']

,heart_rate,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
timestamp,,,,
2021-09-14 22:44:00,55.0,143.0,1.0,0.0


In [50]:
cols = ['heart_rate', 'Glucose Value (mg/dL)']
merged_data[merged_data[cols].notnull().all(axis=1)]

,heart_rate,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
timestamp,,,,
2021-08-15 00:02:00,47.0,0.0,0.0,0.0
2021-08-15 00:04:00,45.0,0.0,0.0,0.0
2021-08-15 00:05:00,49.0,0.0,0.0,0.0
2021-08-15 00:06:00,48.0,0.0,0.0,0.0
2021-08-15 00:07:00,46.0,0.0,0.0,0.0
...,...,...,...,...
2021-09-26 13:11:00,72.0,0.0,0.0,0.0
2021-09-26 13:12:00,84.0,0.0,0.0,0.0
2021-09-26 13:13:00,76.0,0.0,0.0,0.0
